In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install docker
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Test the RISE

In [2]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import platform


In [3]:
def setup_driver():
    chrome_options = Options()
    if platform.system() in ['Windows', 'Darwin']:
        chrome_options.add_experimental_option('detach', True)
    elif platform.system() == 'Linux':
        chrome_options.add_argument('--headless')
    else:
        raise Exception("Unknown OS")
    return webdriver.Chrome(service=ChromiumService(ChromeDriverManager().install()), options=chrome_options)

In [4]:
base_url = "http://localhost:8888"  # Base URL of your JupyterLab server
image_path = "/lab/images/sample-image.jpg"  # Extracted relative path
full_image_url = base_url + image_path
image_url="https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg"

In [5]:
import os
def download_image(image_url, local_img_path):
    response = requests.get(image_url, stream=True)

    if response.status_code == 200:
        os.makedirs(os.path.dirname(local_img_path), exist_ok=True)
        with open(local_img_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"Image downloaded successfully: {local_img_path}")
    else:
        print(f"Failed to download the image: {image_url}")

image_url = 'https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg'
download_image(image_url, 'local_images/sample-image.jpg')


Image downloaded successfully: local_images/sample-image.jpg


In [6]:
def open_file_in_jupyterlab(url, file_name, driver):
    try:
        driver.get(url)
        # Wait for the file browser to be available
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".jp-DirListing-content"))
        )

        # Find the file in the file browser
        file_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{file_name}') and @class='jp-DirListing-itemText']"))
        )

        # Double click on the file to open it
        driver.execute_script("""
            var evt = new MouseEvent('dblclick', {
                bubbles: true,
                cancelable: true,
                view: window
            });
            arguments[0].dispatchEvent(evt);
        """, file_element)

        print(f"File '{file_name}' opened successfully.")
    except TimeoutException:
        print(f"Failed to open file '{file_name}'. File not found or JupyterLab did not load properly.")

In [7]:
def take_screenshot(driver, filename):
    try:
        driver.save_screenshot(filename)
        print(f"Saved screenshot: {filename}")
    except Exception as e:
        print(f"Error taking screenshot: {e}")


In [8]:
def find_element_and_click(driver, xpath):
    try:
        # element = WebDriverWait(driver, 15).until(
        #     EC.element_to_be_clickable((By.XPATH, xpath))
        # )
        element = driver.find_element(By.XPATH, xpath)
        element.click()
        return element
    except Exception as e:
        print(f"Error finding/clicking element: {e}")


## Test 3: Create slides and test RISE

In [9]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell

# Define the cells with their content and metadata
cells = [
    new_markdown_cell("# Example Slidedeck", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**0) Blank**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("We will try to illustrate single-column slides like this one.", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell(source="**1) Image Only**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source='<img src="../../images/sample-image.jpg" width = "100%">', 
                      metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell(source="**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="- Using % of slide to scale the image", 
                      metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell(source="- - - ", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell("**7)Title Slide**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("# Title", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("- a Note that will display in the notes view", metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell("This is the end of the document.", metadata={"slideshow": {"slide_type": "skip"}, "tags": ["remove-cell"]}),
]

In [10]:
def test_rise_slide(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"

    try:
        driver.get(url)
        WebDriverWait(driver, 45)
        take_screenshot(driver, 'main_screenshot.png')

        # Check if on the Launcher page
        try:
            driver.find_element(By.XPATH, "//div[contains(@class, 'lm-TabBar-tabLabel') and contains(@class, 'p-TabBar-tabLabel') and text()='Launcher']")
            print("On the Launcher page")
            find_element_and_click(driver, '//div[@data-category="Notebook"]')
        except NoSuchElementException:
            print("Not on the Launcher page")


        # Create a new notebook object
        nb = new_notebook()

        # Add cells to the notebook
        nb['cells'] = cells

        # Write the notebook to a file
        with open('example_notebook.ipynb', 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)

        print("Notebook created successfully.")

        take_screenshot(driver, 'new_nb_screenshot.png')

        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')
        take_screenshot(driver, 'rise_screenshot.png')

        toolbar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[@class='lm-Widget jp-Toolbar' and @role='navigation']")))
        if toolbar:
            fullscreen_button = toolbar.find_element(By.XPATH, ".//button[@title='Open the slideshow in full screen']")
            fullscreen_button.click()
            is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
            print("Fullscreen button is functioning: ", is_fullscreen)
        else:
            print("Toolbar not found")

        print("RISE Extension is working!")

    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        # Clean up
        driver.switch_to.window(driver.window_handles[0])
        driver.refresh()
        driver.quit()

In [14]:
# replace token with your token
token = "cc2f522441ace86a2bbe4fa85e17f3a488fb181d1538da19"
test_rise_slide(token)

Saved screenshot: main_screenshot.png
Not on the Launcher page
Notebook created successfully.
Saved screenshot: new_nb_screenshot.png
Error finding/clicking element: Message: 
Stacktrace:
0   chromedriver                        0x0000000107e32d28 chromedriver + 4795688
1   chromedriver                        0x0000000107e2a2b3 chromedriver + 4760243
2   chromedriver                        0x0000000107a0388d chromedriver + 407693
3   chromedriver                        0x0000000107a523d9 chromedriver + 730073
4   chromedriver                        0x0000000107a52591 chromedriver + 730513
5   chromedriver                        0x0000000107a987e4 chromedriver + 1017828
6   chromedriver                        0x0000000107a79ccd chromedriver + 892109
7   chromedriver                        0x0000000107a95b2b chromedriver + 1006379
8   chromedriver                        0x0000000107a79a73 chromedriver + 891507
9   chromedriver                        0x0000000107a44143 chromedriver + 67206